## Imports

In [ ]:
import yaml
import pandas as pd
from kedro_datasets.pandas import GBQTableDataSet
from kedro.io import PartitionedDataSet

## Loads kedro yml data

In [ ]:
with open('../conf/base/catalog.yml') as f:
    # use safe_load instead load
    catalog = yaml.safe_load(f)

## Query using bigquery API for query debugging

In [ ]:
# bq_query_cat_entry = catalog['bq_test']
# project_id = bq_query_cat_entry['project']

# Recovers contents of ARRAYS nested inside STRUCTS
# sql = """SELECT element.array_element
#     FROM bigquery-public-data.gbif.occurrences,
#     UNNEST(issue.array) AS element
#     LIMIT 20"""

# Shows the distinct values of the issue column, which is an ARRAY nested inside a STRUCT
# sql = """SELECT distinct element.array_element
#     FROM bigquery-public-data.gbif.occurrences,
#     UNNEST(issue.array) AS element"""

# Counts the number of observations per species and orders it from high to low
# sql = """ SELECT species,
#   count(species) as cnt
#   FROM bigquery-public-data.gbif.occurrences
#   group by species
#   order by cnt desc"""

# Counts the number of observations per species and orders it from high to low
# sql = """ SELECT species,
#   count(species) as cnt
#   FROM bigquery-public-data.gbif.occurrences
#   group by species
#   order by cnt desc"""

# Gets top 20 rows of all data
# sql = """ SELECT gbifid
#     ,datasetkey
#     ,occurrenceid
#     ,kingdom
#     ,phylum
#     ,class
#     ,`order`
#     ,family
#     ,genus
#     ,species
#     ,infraspecificepithet
#     ,taxonrank
#     ,scientificname
#     ,verbatimscientificname
#     ,verbatimscientificnameauthorship
#     ,countrycode
#     ,locality
#     ,stateprovince
#     ,occurrencestatus
#     ,individualcount
#     ,publishingorgkey
#     ,decimallatitude
#     ,decimallongitude
#     ,coordinateuncertaintyinmeters
#     ,coordinateprecision
#     ,elevation
#     ,elevationaccuracy
#     ,depth
#     ,depthaccuracy
#     ,eventdate
#     ,day
#     ,month
#     ,year
#     ,taxonkey
#     ,specieskey
#     ,basisofrecord
#     ,institutioncode
#     ,collectioncode
#     ,catalognumber
#     ,recordnumber
#     ,identifiedby
#     ,dateidentified
#     ,license
#     ,rightsholder
#     ,recordedby
#     ,typestatus
#     ,establishmentmeans
#     ,lastinterpreted
#     ,mediatype
#     ,issue
# FROM bigquery-public-data.gbif.occurrences
# LIMIT 20 """

# 
# sql = """SELECT gbifid
#     ,kingdom
#     ,phylum
#     ,class
#     ,`order`
#     ,family
#     ,genus
#     ,species
#     ,countrycode
#     ,occurrencestatus
#     ,individualcount
#     ,decimallatitude
#     ,decimallongitude
#     ,eventdate
#     ,day
#     ,month
#     ,year
#     ,basisofrecord 
#     ,element.array_element as issue
#     FROM bigquery-public-data.gbif.occurrences
#     CROSS JOIN UNNEST(issue.array) AS element
#     WHERE species = 'Anas platyrhynchos' AND eventdate > '2020-01-01'
#     """

# data_set = GBQQueryDataSet(sql, project=project_id)
# df = data_set.load()

## Loads csv with data downladed from bigquery

In [ ]:
catalog_entry = 'species_data'
path = '..//' + catalog[catalog_entry]['path']
dataset = catalog[catalog_entry]['dataset']

data_set = PartitionedDataSet(
       path=path,
       dataset=dataset,
   )

print(dataset)

In [ ]:
loaded = data_set.load()

combine_all = pd.DataFrame()
for partition_id, partition_load_func in loaded.items():
       partition_data = partition_load_func()
       combine_all = pd.concat(
           [combine_all, partition_data], ignore_index=True, sort=True
       )

In [ ]:
combine_all.info()